In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import os
from splinter import Browser
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser("chrome", **executable_path, headless=False)

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection  from Eli's Stu_Reddit_Scraper for 12.2.4 that he slacked to us at 7:51 on 3/16
db = client.mars_news_db
collection = db.articles
print(f'collection; {collection}')


collection; Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_news_db'), 'articles')


Step 1 - Scraping:  NASA Mars News

In [4]:
# Set-up path and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)
# Scrape Mars News
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

# Add try/except for error handling
try:
    slide_elem = news_soup.select_one('div.list_text')
    # Use the parent element to find the first 'a' tag and save it as 'news_title'
    news_title = slide_elem.find("div", class_="content_title").get_text()
    # Use the parent element to find the paragraph text
    news_p = slide_elem.find("div", class_="article_teaser_body").get_text()

except Exception as e:
    print()
    print(f'error: {e}')
    print()
    browser.quit()

browser.quit()
print(f'news_title: {news_title}')
print(f'news_p: {news_p}')




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


news_title: NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
news_p: Learn more about the agency's next Red Planet mission during a live event on June 17.


Step 1 - Scraping:  JPL Mars Space Images - Featured Image

In [5]:
# Step 1 - Scraping:  JPL Mars Space Images - Featured Mars Image

# Set-up path and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://spaceimages-mars.com/'

# Retrieve page with the visit module
browser.visit(url)
html = browser.html
time.sleep(1)

try: 
    # Click on "FULL IMAGE" button
    browser.links.find_by_partial_text('FULL IMAGE').click()
    time.sleep(1)

    # Create BeautifulSoup object; parse with 'html.parser'
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup.prettify())

    # Find full size image
    fade_in_image = soup.find_all("img", "headerimage fade-in")
    featured_image_url=url + fade_in_image[0]['src']
    browser.quit()


except Exception as e:
    print(f'error: {e}')
    browser.quit()
print(f'featured_image_url: {featured_image_url}')




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


featured_image_url: https://spaceimages-mars.com/image/featured/mars1.jpg


Step 1 - Scraping:  JPL Mars Space Images - Mars Facts

In [6]:
    # Step 1 - Scraping:  JPL Mars Space Images - Mars Facts

    # Set-up path and browser
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser("chrome", **executable_path, headless=False)
    url = 'https://galaxyfacts-mars.com/'

    try:
        tables = pd.read_html(url)
        time.sleep(1)
        # Get Dataframe of first table
        # print(f'($$$$$$$$$$$$$$$$$$$$$$$$$$$$$Tables in mars_facts is: {tables}')
        mars_facts_df = tables[0]
        mars_facts_df.rename(columns={0: "Description", 1: "Mars", 2: "Earth"}, inplace=True)
        mars_facts_df=mars_facts_df.set_index("Description")
        html_table = mars_facts_df.to_html()
        browser.quit()

    except Exception as e:
        print()
        print("?????????????????????????? Exception in mars_facts ????????????????????????????????")
        print(f'error: {e}')
        print()
        browser.quit()
print(f'html_table: {html_table}')



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


html_table: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


Step 1 - Scraping:  Mars Hemispheres

In [7]:
# Converted my Jupyter Notebook mission_to_mars.ipynb into to scrape_mars.py
# then bloked off key funtions 
# Dependencies
# Started with scrape_costa.py basics and added more over time

from splinter import Browser
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

import requests
import pymongo
import os
import pandas as pd


# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection  from Eli's Stu_Reddit_Scraper for 12.2.4 that he slacked to us at 7:51 on 3/16
db = client.mars
collection = db.articles
print(f'collection; {collection}')


# Step 1 - Scraping:  Mars Hemispheres

# Set-up path and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://marshemispheres.com/'

# Retrieve page with the visit module
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())
results = soup.find_all('div', class_='item')

i = 0
titles = {}
img_urls = {}
Enhanced_dict= {}
mydict ={}
hemisphere_image_urls = []


# Next look for <h3> (title) and title link to click while on intial page
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = results[i].find('h3').text
        titles[i]=title

        # Run only if title is available
        if (title != ''):

            # Click on proper enhanced image button
            browser.links.find_by_partial_text(title).click()
            time.sleep(1)

            # Next look for 'downloads to find the actual link
            html = browser.html
            # print(html)

            # Create BeautifulSoup object; parse with 'html.parser'; find description iterables
            enhanced_soup = BeautifulSoup(html, 'html.parser')
            enhanced_results = enhanced_soup.find_all('div', class_='downloads')

            # Loop through returned results
            for enhanced_result in enhanced_results:
                # Error handling
                try:
                    link = enhanced_result.a['href']
                    img_url=url + link
                except Exception as e:
                    print(e)
            hemisphere_image_urls.append(
                {
                    'title': title,
                    'img_url': img_url,
                }
            )


            # Click on back button to return to home page
            browser.links.find_by_partial_text('Back').click()
            time.sleep(1)

    except Exception as e:
        browser.quit()
        print(f'error: {e}')

    i+=1

browser.quit()
print (f'hemisphere_image_urls: {hemisphere_image_urls}')




====== WebDriver manager ======


collection; Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars'), 'articles')


Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


hemisphere_image_urls: [{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]
